In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

In [8]:
insurance=pd.read_csv("C:/Users/ACER/OneDrive/Jupyter Files/Machine-Learning-with-R-datasets/insurance.csv")
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [24]:
ct = make_column_transformer((MinMaxScaler(),['age','bmi','children']),
                            (OneHotEncoder(handle_unknown='ignore'),['sex','smoker','region']))

X = insurance.drop(columns='charges')
y = insurance['charges']

X.shape, y.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train.shape, X_test.shape

ct.fit(X_train)

X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train.shape, X_train_normal.shape

((1137, 6), (1137, 11))

In [ ]:
tf.random.set_seed(42)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100,input_shape=[11],activation='relu',kernel_initializer='normal',bias_initializer='random_normal'),
    tf.keras.layers.Dense(100,activation='relu',kernel_initializer='normal',bias_initializer='random_normal'),
    tf.keras.layers.Dense(50,activation='exponential',kernel_initializer='normal',bias_initializer='random_normal'),
    tf.keras.layers.Dense(1)
])

model.compile(
    loss=tf.keras.losses.mae,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["mae"]
)

print(model.summary())
history = model.fit(X_train_normal, y_train, epochs=1000, verbose = 1, callbacks=[callback])

In [ ]:
pd.DataFrame(history.history).plot()
plt.ylabel("loss")
plt.xlabel("epoch")

In [ ]:
model.evaluate(X_test_normal,y_test)

In [ ]:
y_pred = model.predict(X_test_normal)
tf.squeeze(y_pred) - y_test